In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql import functions as F
from pyspark.sql import Window

spark = (SparkSession.builder
            .appName('PySparkMinIO')
            .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.3.1,"
                                            "com.amazonaws:aws-java-sdk-bundle:1.12.709")
            .config("spark.hadoop.fs.s3a.access.key", "andreyolv")
            .config("spark.hadoop.fs.s3a.secret.key", "andreyolv")
            .config("spark.hadoop.fs.s3a.endpoint", "http://minio.minio:9000")
            .config("spark.hadoop.fs.s3a.path.style.access", True)
            .config("spark.hadoop.fs.s3a.fast.upload", True)
            .config("spark.hadoop.fs.s3a.multipart.size", 104857600)
            .config("fs.s3a.connection.maximum", 100)
            .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
            .getOrCreate()
        )

In [ ]:
# Create a schema for your dataframe
schema = StructType([
    StructField("name", StringType(), True),
    StructField("email", StringType(), True),
    StructField("address", StringType(), True),
    StructField("phone_number", StringType(), True)
])

# Use the Faker library to generate data for your dataframe
data = [
    ('Andrey', 'andrey@gmail.com', 'Rua dos Canários', '4199991111'),
    ('Joao', 'jaozinho@gmail.com', 'Rua dos Pardais', '4199992222'),
    ('Amora', 'amorinha@gmail.com', 'Rua das Andorinhas', '4199993333'),
    ('Lucy', 'lucymara@gmail.com', 'Rua dos Papagaios', '4199994444'),]

# Create a PySpark dataframe from your data and schema
df = spark.createDataFrame(data, schema)

# Show your dataframe
df.show()

## Save Parquet

In [ ]:
s3_location = 's3a://lakehouse/parquet'

In [ ]:
df.write.mode('overwrite').option('header', True).parquet(s3_location)

## Load Parquet

In [ ]:
spark.read.format("parquet").load(s3_location).toPandas()